# Analisis de Egresos Hospitalarios

In [25]:
import importlib

import pandas as pd
import numpy as np
import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns

import lectura_archivos as fr
import calculo_metricas as aux

from functools import reduce

sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

importlib.reload(fr)
importlib.reload(aux)


<module 'calculo_metricas' from 'c:\\Users\\ppizarro\\Desktop\\Javier Rojas\\Javier\\trabajo\\SSMO\\EPH\\modulo_gestion\\1.analisis_problemas_de_salud\\egresos-hospitalarios\\calculo_metricas.py'>

In [26]:
HOSPITAL_A_ANALIZAR = 112103  # Este es el Torax


## Lectura de archivos

- Se leen todos los archivos que estan en input/utf-8. Los archivos estan separados por ;
- Se dejan solamente los egresos de los diagnosticos que **SI** tenga el Tórax
- Se cambia la codificacion de INTERV_Q de 2 (No, 1 es SI) a 0.
- Se cambia la codificacion de CONDICION_EGRESO (orig 1 = Vivo, 2 = Fallecido) de 1 -> 0 y 2 -> 1
- Se cambia la codificacion de GLOSA_REGION_RESIDENCIA {
                        "Del Libertador B. O'Higgins": "del Libertador General Bernardo O'Higgins",
                        "De Aisén del Gral. C. Ibáñez del Campo": "Aysén del General Carlos Ibáñez del Campo",
                    }

In [27]:
df = fr.leer_archivos(HOSPITAL_A_ANALIZAR)

# Analisis de Egresos Hospitalarios

Se calculan las siguientes metricas:

- Número de Egresos (n_egresos)
- Dias estada totales (dias_estada_totales)
- Cantidad de Intervenciones Quirúrgicas (n_int_q)
- Cantidad de Muertes (n_muertos)

Todas estas metricas estan calculadas a nivel de Año, Establecimiento de Salud y Diagnóstico. Por
ejemplo:

***"En el año 2019, el Hospital San José (código XXXXXXX) tuvo XX egresos, con XX dias estada_promedio,
con XX intervenciones_quirúrgicas y XX muertos para el diagnóstico A24.4."***

Este análisis se realizó entre 2001 y 2020 para todos los hospitales/clínicas de Chile.

In [28]:
AGRUPACION = [
    "ANO_EGRESO",
    "ESTABLECIMIENTO_SALUD",
    "GLOSA_ESTABLECIMIENTO_SALUD",
    "DIAG1",
]

metricas = aux.obtener_metricas_egresos(df, AGRUPACION).collect(streaming=True)


# Analisis de Estratos

In [29]:
dict_estratos = aux.obtener_diccionario_estratos(df, HOSPITAL_A_ANALIZAR)
cie = fr.leer_cie()


In [30]:
variables_a_rankear = ["n_egresos"]
subgrupo_del_ranking = ["ANO_EGRESO", "DIAG1"]
unir_tablas_por = AGRUPACION

ORDEN_COLS = [
    "ANO_EGRESO",
    "ESTABLECIMIENTO_SALUD",
    "GLOSA_ESTABLECIMIENTO_SALUD",
    "Capítulo",
    "Sección",
    "Categoría",
    "Descripción",
    "DIAG1",
    "n_egresos",
    "dias_estada_totales",
    "n_int_q",
    "n_muertos",
    "ranking_nacionales_n_egresos",
    "total_nacionales_n_egresos",
    "%_nacionales_n_egresos",
    "ranking_publicos_n_egresos",
    "total_publicos_n_egresos",
    "%_publicos_n_egresos",
    "ranking_privados_n_egresos",
    "total_privados_n_egresos",
    "%_privados_n_egresos",
    "ranking_grd_n_egresos",
    "total_grd_n_egresos",
    "%_grd_n_egresos",
    "ranking_interno_n_egresos",
    "total_interno_n_egresos",
    "%_interno_n_egresos",
]


In [31]:
dict_resultados_estratos = aux.obtener_resumen_por_estratos(
    metricas, dict_estratos, variables_a_rankear, subgrupo_del_ranking
)

resultado = reduce(aux.left_join_consecutivo, dict_resultados_estratos.values())
resultado_unido = resultado.join(cie, how="left", on="DIAG1")
resultado_unido = resultado_unido.select(ORDEN_COLS)

In [32]:
resultados_torax = resultado_unido.filter(pl.col("ESTABLECIMIENTO_SALUD") == HOSPITAL_A_ANALIZAR)
torax = resultados_torax.to_pandas()
torax.to_csv(
    "output/ranking_torax_pais/posiciones_torax_polars.csv",
    sep=";",
    decimal=".",
    encoding="latin-1",
    index=False,
)


In [33]:
diags_mas_relevantes = resultados_torax.filter(
    (pl.col("ranking_nacionales_n_egresos") == 1) & (pl.col("n_int_q") > 0)
).sort(by=["ANO_EGRESO", "n_egresos", "n_int_q"], descending=True)

veinte_mas_relevantes_por_anio = diags_mas_relevantes.groupby(["ANO_EGRESO"]).head(20)


# Analisis Sociodemográfico

In [34]:
desglose_sociodemografico = [
    "ANO_EGRESO",
    "REGION_RESIDENCIA",
    "GLOSA_REGION_RESIDENCIA",
    "region_pais",
    "COMUNA_RESIDENCIA",
    "GLOSA_COMUNA_RESIDENCIA",
    "comuna_region_pais",
    "DIAG1",
]

variables_a_analizar = [
    "DIAG1",
    "SEXO",
    "EDAD_CATEGORIA",
    "PUEBLO_ORIGINARIO",
    "GLOSA_PAIS_ORIGEN",
    "PREVISION",
    "BENEFICIARIO",
]

In [35]:
df_socio = fr.obtener_df_inicial_sociodemografico()


In [36]:
dict_resultados = {}

for variable in variables_a_analizar:
    agrupacion_total = desglose_sociodemografico.copy()
    if variable != "DIAG1":
        agrupacion_total += [variable]

    resultado = (
        df_socio.groupby(agrupacion_total)
        .agg(pl.col(variable).count().alias("conteo"))
        .sort(["ANO_EGRESO", "DIAG1", "conteo"], descending=True)
        .with_columns(
            pl.concat_str(pl.col(desglose_sociodemografico), separator="-").alias("llave_id")
        )
    )

    dict_resultados[variable] = resultado


In [37]:
import xlsxwriter

with xlsxwriter.Workbook("output/distribucion_sociodemografica/distribucion_polars.xlsx") as f:
    for var, df in dict_resultados.items():
        df.write_excel(workbook=f, worksheet=var)


# Analisis de cantidad de Camas a ocupar

- Aqui se quiere ver cuantas camas son necesarias por diagnostico
- El periodo que se quiere obtener es cada mes.
- Por lo tanto, se deben obtener la cantidad de dias estada totales por mes. Luego, dividir
esa cantidad por los dias del periodo.

In [38]:
agrupacion_2 = [
    "ANO_EGRESO",
    "MES_EGRESO",
    "ESTABLECIMIENTO_SALUD",
    "GLOSA_ESTABLECIMIENTO_SALUD",
    "DIAG1",
]

with pl.StringCache():
    df = pl.scan_csv("input/utf-8/*.csv", separator=";", dtypes=fr.DICT_VARIABLES).with_columns(
        pl.col("FECHA_EGRESO").dt.month().alias("MES_EGRESO")
    )

    prueba = aux.obtener_metricas_egresos(df, agrupacion_2).filter(
        pl.col('ESTABLECIMIENTO_SALUD') == 112103
    )
    resultado = prueba.collect(streaming=True)

In [39]:
resultado.write_csv('output/cantidad_camas/metricas_por_mes.csv', separator=';')